In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"


In [2]:
from geomloss import SamplesLoss

# p = 1 is Wasserstein-1 distance (also called EMD)
# p = 2 is Wasserstein-2 distance 


# p = 1, and blur = 0.0001 are the hyperparameters we use in this work.
OT_LOSS = SamplesLoss(loss="sinkhorn", p=1, blur=0.0001)


Here is an example of how to use geomloss package.

In [ ]:

import torch

# Create some large point clouds in 3D
x = torch.randn(100000, 3, requires_grad=True).cuda()
y = torch.randn(200000, 3).cuda()

L = OT_LOSS(x, y)  # By default, use constant weights = 1/number of samples
g_x, = torch.autograd.grad(L, [x])  # GeomLoss fully supports autograd!

To put this into context of this work, let's compute loss on the soft body.

In [3]:
import os
from pathlib import Path

from mpm.hand import HandEnv
from tools import CN
from tools.config import merge_inputs

def get_default_cfg(cfg_path, sim_cfg=None):
    assert os.path.exists(cfg_path), "config file does not exist!"

    cfg = CN(new_allowed=True)
    cfg.env_name = Path(cfg_path).stem
    cfg.merge_from_file(cfg_path)

    if sim_cfg is not None:
        cfg.defrost()
        cfg.SIMULATOR = merge_inputs(cfg.SIMULATOR, **sim_cfg)
        cfg.freeze()

    return cfg


LOADING....
Loaded with 0.00063323974609375 secs.
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
from mpm import make

env = make(env_name='folding', sim_cfg={'max_steps': 4400})
init_state = env.init_state


{'h': 0.01, 'r': 0.16, 'init_pos': (0.5, 0.061875, 0.6), 'n_particles': 10000}
GPU 0/1 memory: free=39.074, total=47.536
init renderer


In [6]:
import torch

current_shape_particles = env.simulator.get_x(0, device='cuda')
target_shape_particles = torch.randn(200000, 3).cuda()

L = OT_LOSS(current_shape_particles, target_shape_particles)  # By default, use constant weights = 1/number of samples

In [7]:
L.item()

1.670701503753662